In [6]:
import numpy as np
import pandas as pd
import os,sys,glob
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
STUDY_PATH=os.path.join('/NAS','coolio','protocoles','CINOCIS')
SCRIPT_PATH=os.path.join(STUDY_PATH,'scripts')
BIDS_PATH=os.path.join(STUDY_PATH,'BIDS')

In [4]:
participants_file = os.path.join(BIDS_PATH, 'participants.tsv')
participants_df = pd.read_csv(participants_file, sep='\t')
print(participants_df)

    participant_id sex
0          sub-001   F
1          sub-002   F
2          sub-003   F
3          sub-004   F
4          sub-005   M
..             ...  ..
116        sub-129   M
117        sub-130   F
118        sub-131   F
119        sub-132   F
120        sub-133   F

[121 rows x 2 columns]


In [8]:
# Drop duplicate participant_id entries
unique_participants_df = participants_df.drop_duplicates(subset='participant_id')

# Write the unique participants to a new TSV file
unique_participants_file = os.path.join(BIDS_PATH, 'unique_participants.tsv')
unique_participants_df.to_csv(unique_participants_file, sep='\t', index=False)

print(f"Unique participants written to {unique_participants_file}")

Unique participants written to /NAS/coolio/protocoles/CINOCIS/BIDS/unique_participants.tsv


In [5]:
import glob
import csv
from datetime import datetime

# Define the path pattern for the log files
log_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'sub-*', 'ses-*', 'logs', 'proc_structural*.txt')

# Define the output CSV file
output_csv_file = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'micapipe_processed_sub.csv')

# Open the CSV file in append mode
with open(output_csv_file, mode='a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Iterate over all log files matching the pattern
    for log_file in glob.glob(log_files_pattern):
        with open(log_file, 'r') as file:
            content = file.read()
            if "Status          : COMPLETED" in content:
                # Extract SUBJ and SES from the file path
                parts = log_file.split(os.sep)
                subj = parts[-4].split('-')[1]
                ses = parts[-3].split('-')[1]
                
                # Define the row to be added
                row = [subj,f" {ses}", 'proc_structural', ' COMPLETED', ' 09/09', ' romain', ' sim', 
                       datetime.now().strftime('%a. %d %b %Y %H:%M:%S CET'), ' 67.567', ' LOCAL', " v0.2.2 'Northern flicker'"]
                
                # Write the row to the CSV file
                csv_writer.writerow(row)

print(f"Log processing completed and results written to {output_csv_file}")

Log processing completed and results written to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/micapipe_v0.2.0/micapipe_processed_sub.csv


In [7]:
# Define the path pattern for the new log files
log_files_pattern_post_structural = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'sub-*', 'ses-*', 'logs', 'post_structural*.txt')

# Open the CSV file in append mode
with open(output_csv_file, mode='a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Iterate over all log files matching the new pattern
    for log_file in glob.glob(log_files_pattern_post_structural):
        with open(log_file, 'r') as file:
            content = file.read()
            if "Status          : COMPLETED" in content:
                # Extract SUBJ and SES from the file path
                parts = log_file.split(os.sep)
                subj = parts[-4].split('-')[1]
                ses = parts[-3].split('-')[1]
                
                # Define the row to be added
                row = [subj, f"{ses}", 'post_structural', 'COMPLETED', '07/07', 'romain', 'sim', 
                       datetime.now().strftime('%a. %d %b %Y %H:%M:%S CET'), '67.567', 'LOCAL', "v0.2.2 'Northern flicker'"]
                
                # Write the row to the CSV file
                csv_writer.writerow(row)

print(f"Log processing completed and results written to {output_csv_file}")

Log processing completed and results written to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/micapipe_v0.2.0/micapipe_processed_sub.csv


In [6]:
# Define the path pattern for the new log files
log_files_pattern_surf = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'sub-*', 'ses-*', 'logs', 'proc_surf_*.txt')

# Open the CSV file in append mode
with open(output_csv_file, mode='a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Iterate over all log files matching the new pattern
    for log_file in glob.glob(log_files_pattern_surf):
        with open(log_file, 'r') as file:
            content = file.read()
            if "Status          : COMPLETED" in content:
                # Extract SUBJ and SES from the file path
                parts = log_file.split(os.sep)
                subj = parts[-4].split('-')[1]
                ses = parts[-3].split('-')[1]
                
                # Define the row to be added
                row = [subj, f"{ses}", 'proc_surf-fastsurfer', 'COMPLETED', '01/01', 'romain', 'sim', 
                       datetime.now().strftime('%a. %d %b %Y %H:%M:%S CET'), '67.567', 'LOCAL', "v0.2.2 'Northern flicker'"]
                
                # Write the row to the CSV file
                csv_writer.writerow(row)

print(f"Log processing completed and results written to {output_csv_file}")

Log processing completed and results written to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/micapipe_v0.2.0/micapipe_processed_sub.csv


 # Suppression des composantes connexes inférieurs à 4mm

In [10]:
import nibabel as nib
import scipy.ndimage as ndimage

# Define the path pattern for the lesion segmentation files
lesionseg_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3', 'sub-*', 'ses-M*', 'sub-*lesionseg.nii.gz')

# Iterate over all lesion segmentation files matching the pattern
for lesionseg_file in glob.glob(lesionseg_files_pattern):
    # Load the NIfTI file
    img = nib.load(lesionseg_file)
    data = img.get_fdata()
    
    # Get voxel dimensions (in mm)
    voxel_dims = img.header.get_zooms()
    voxel_volume = np.prod(voxel_dims)
    
    # Label connected components
    labeled_array, num_features = ndimage.label(data)
    
    # Count the number of voxels for each connected component
    voxel_counts = ndimage.sum(data, labeled_array, range(1, num_features + 1))
    
    print(f"File: {lesionseg_file}")
    for i, count in enumerate(voxel_counts, start=1):
        volume_mm3 = count * voxel_volume
        print(f"  Component {i}: {count} / {volume_mm3:.2f} mm³")
        # Threshold volume in mm³
        threshold_volume_mm3 = 4.0
        
        # Create a mask for components larger than the threshold
        mask = np.zeros_like(data, dtype=bool)
        for i, count in enumerate(voxel_counts, start=1):
            volume_mm3 = count * voxel_volume
            if volume_mm3 >= threshold_volume_mm3:
                mask |= (labeled_array == i)
        
        # Create a new NIfTI image with the thresholded data
        thresholded_data = data * mask
        thresholded_img = nib.Nifti1Image(thresholded_data, img.affine, img.header)
        
        # Save the new image
        thresholded_file = lesionseg_file.replace('.nii.gz', '_thres4.nii.gz')
        nib.save(thresholded_img, thresholded_file)
        
        print(f"Thresholded image saved to {thresholded_file}")

File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg.nii.gz
  Component 1: 6.0 / 4.71 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_thres4.nii.gz
  Component 2: 47.0 / 36.91 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_thres4.nii.gz
  Component 3: 148.0 / 116.22 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_thres4.nii.gz
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_cut_lesionseg.nii.gz
  Component 1: 81.0 / 63.61 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub

# fusionner les deux volumes (cervical et thoracique coupé)

In [13]:
import nibabel as nib
import numpy as np

# Chemins des fichiers NIfTI à fusionner
file1 = '//NAS/coolio/protocoles/CINOCIS/BIDS/sub-001/ses-M3/anat/sub-001_ses-M3_acq-801T2W_SPAIR2D_T2w.nii.gz'
file2 = '/NAS/coolio/protocoles/CINOCIS/BIDS/sub-001/ses-M3/anat/sub-001_ses-M3_acq-802T2W_SPAIRtho2D_T2w_cut.nii.gz'

# Charger les deux volumes
img1 = nib.load(file1)
img2 = nib.load(file2)

# Extraire les données des volumes
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(f"Dimensions du volume 1 : {data1.shape}")
print(f"Dimensions du volume 2 : {data2.shape}")

# Vérifier que les dimensions des volumes sont compatibles pour la fusion
if data1.shape != data2.shape:
    raise ValueError("Les dimensions des volumes ne correspondent pas et ne peuvent pas être fusionnées.")

# Fusionner les volumes le long de l'axe z (par exemple)
fused_data = np.concatenate((data1, data2), axis=2)

# Créer une nouvelle image NIfTI avec les données fusionnées
fused_img = nib.Nifti1Image(fused_data, img1.affine, img1.header)

# Chemin pour enregistrer le volume fusionné
fused_file = '/NAS/coolio/protocoles/CINOCIS/BIDS/sub-001/ses-M3/anat/sub-001_ses-M3_acq-802T2W_SPAIRtho2D_T2w_fused.nii.gz'

# Enregistrer le volume fusionné
nib.save(fused_img, fused_file)

print(f"Volume fusionné enregistré à {fused_file}")

Dimensions du volume 1 : (512, 512, 12)
Dimensions du volume 2 : (12, 512, 451)


ValueError: Les dimensions des volumes ne correspondent pas et ne peuvent pas être fusionnées.

In [23]:
# Define the path pattern for the labeled segmentation files
labeled_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused', 'sub-*', 'ses-M*', 'sub-*_ses-*_acq*_T2w_fus_seg_labeled.nii.gz')

# Iterate over all labeled segmentation files matching the pattern
for labeled_file in glob.glob(labeled_files_pattern):
    # Load the NIfTI file
    img = nib.load(labeled_file)
    data = img.get_fdata()
    
    # Label connected components
    num_features = ndimage.unique(data)
    
    print(f"File: {labeled_file}")
    print(f"  Number of labels: {num_features}")

File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused/sub-078/ses-M3/sub-078_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz
  Number of labels: 1
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused/sub-078/ses-M60/sub-078_ses-M60_acq-1302T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz
  Number of labels: 3
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused/sub-126/ses-M3/sub-126_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz
  Number of labels: 1
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused/sub-126/ses-M60/sub-126_ses-M60_acq-302T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz
  Number of labels: 2
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused/sub-107/ses-M3/sub-107_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz
  Number of labels: 1
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused/sub-107/ses-M60/sub-107_ses-M60_acq-302T2W_SPAIRt